In [12]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [14]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [15]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [16]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent

model = ChatOllama(model="lfm2.5-thinking")

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [17]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='66b647d2-dd94-47c5-a0d1-5ca710d125ff'),
              AIMessage(content="The **langchain-mcp-adapters** library is part of the LangChain ecosystem designed to enhance model composition and integration capabilities. It allows users to adapt and combine models (e.g., from different frameworks like TensorFlow, PyTorch, or custom models) into cohesive workflows within LangChain. This facilitates seamless integration of diverse models into a unified pipeline, enabling tasks like model selection, training, or deployment within a single workflow. It supports modularity and flexibility, making it useful for customizing or extending LangChain's functionality. For specific implementation details, consult the LangChain documentation or community resources. Let me know if you'd like further clarification!", additional_kwargs={}, response_metadata={'model': 'lfm2.5-

## Online MCP

In [21]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "ls",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

  + Exception Group Traceback (most recent call last):
  |   File "/Users/tomaszgaluszka/.pyenv/versions/lnd_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/76/l6f865dn69ddcxry4_d4l6br0000gn/T/ipykernel_43768/1663329054.py", line 14, in <module>
  |     tools = await client.get_tools()
  |             ^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/tomaszgaluszka/.pyenv/versions/lnd_env/lib/python3.13/site-packages/langchain_mcp_adapters/client.py", line 197, in get_tools
  |     tools_list = await asyncio.gather(*load_mcp_tool_tasks)
  |                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/tomaszgaluszka/.pyenv/versions/lnd_env/lib/python3.13/site-packages/langchain_mcp_adapters/tools.py", line 478, in load_mcp_tools
  |     async with create_session(
  |                ~~~~~~~~~~~~~~^
  |         connection, mcp_callbacks=mcp_callba

In [10]:
agent = create_agent(
    model=model,
    tools=tools,
)

In [11]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='ca9d0769-57c2-4512-aac3-49ac3929abb1'),
              AIMessage(content='I don\'t have real-time access to current time information. However, you can check the current time using:\n\n1. Your device\'s clock/app  \n2. A search engine (e.g., "What time is it today?")  \n3. A world clock website  \n\nIf you\'d like, I can help you find a reliable source for this information! Just let me know your location or preferred time zone.', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T08:47:26.977863Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2711434042, 'load_duration': 39688125, 'prompt_eval_count': 72, 'prompt_eval_duration': 99272000, 'eval_count': 343, 'eval_duration': 2523821498, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a3b-7a67-7233-b607-5ef2c789a3e9-0', tool_calls=[], in